In [1]:
import nltk, string
from nltk.corpus import cess_esp
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *
from nltk import Tree

In [2]:
#***************************************************************************************
# 1. Cargamos el corpus de textos
#***************************************************************************************

print("\n\n1. Cargamos el corpus de prueba: \n")

corpus = "Ponme un pincho de tortilla, quiero una pizza margherita, Quiero una hamburguesa,Podría ser un bocadillo de calamares, Una ración de patatas, Quería pedir dos cheeseburger,una ración de jamón ibérico, Dos pizzas medianas barbacoa, Un kebab mixto,Chili con queso,por favor"
print(corpus)



1. Cargamos el corpus de prueba: 

Ponme un pincho de tortilla, quiero una pizza margherita, Quiero una hamburguesa,Podría ser un bocadillo de calamares, Una ración de patatas, Quería pedir dos cheeseburger,una ración de jamón ibérico, Dos pizzas medianas barbacoa, Un kebab mixto,Chili con queso,por favor


In [3]:
sents = cess_esp.tagged_sents()

#Metemos en el conjunto de entrenamiento el 90% de las frases, y el restante 10% en el conjunto de test

training = []
test = []
for i in range(len(sents)):
    if i % 10:
        training.append(sents[i])
    else:
        test.append(sents[i])

#Entrenamos los 4 posibles taggers con el corpus CESS_ESP

unigram_tagger = UnigramTagger(training)
bigram_tagger = BigramTagger(training, backoff=unigram_tagger)
trigram_tagger = TrigramTagger(training, backoff=bigram_tagger)
hmm_tagger = HiddenMarkovModelTagger.train(training)

print ('Acierto con unigramas:', unigram_tagger.evaluate(test)*100)
print ('Acierto con bigramas:', bigram_tagger.evaluate(test)*100)
print ('Acierto con trigramas:', trigram_tagger.evaluate(test)*100)
print ('Acierto con HMMs:', hmm_tagger.evaluate(test)*100)

Acierto con unigramas: 87.65970871234029
Acierto con bigramas: 89.42636311057363
Acierto con trigramas: 89.40007361059993
Acierto con HMMs: 89.88905831011094


In [5]:
# Dividimos el corpus de prueba en frases

sentences = nltk.tokenize.sent_tokenize(corpus)
print ("\n\n2. Frases:",sentences)



2. Frases: ['Ponme un pincho de tortilla, quiero una pizza margherita, Quiero una hamburguesa,Podría ser un bocadillo de calamares, Una ración de patatas, Quería pedir dos cheeseburger,una ración de jamón ibérico, Dos pizzas medianas barbacoa, Un kebab mixto,Chili con queso,por favor']


In [6]:
# Tokenizamos las frases

tokens = nltk.word_tokenize(corpus)
print ("\n\n3. Tokens:",tokens)



3. Tokens: ['Ponme', 'un', 'pincho', 'de', 'tortilla', ',', 'quiero', 'una', 'pizza', 'margherita', ',', 'Quiero', 'una', 'hamburguesa', ',', 'Podría', 'ser', 'un', 'bocadillo', 'de', 'calamares', ',', 'Una', 'ración', 'de', 'patatas', ',', 'Quería', 'pedir', 'dos', 'cheeseburger', ',', 'una', 'ración', 'de', 'jamón', 'ibérico', ',', 'Dos', 'pizzas', 'medianas', 'barbacoa', ',', 'Un', 'kebab', 'mixto', ',', 'Chili', 'con', 'queso', ',', 'por', 'favor']


In [7]:
# Obtenemos los tags con los 4 métodos

tagged1 = unigram_tagger.tag(tokens)
print ("TAGGER UNIGRAMA:",tagged1)
print ("________________________")
tagged2 = bigram_tagger.tag(tokens)
print ("TAGGER BIGRAMA:",tagged2)
print ("________________________")
tagged3 = trigram_tagger.tag(tokens)
print ("TAGGER TRIGRAMA:",tagged3)
print ("________________________")
tagged4 = hmm_tagger.tag(tokens)
print ("TAGGER HMMs:",tagged4)

TAGGER UNIGRAMA: [('Ponme', None), ('un', 'di0ms0'), ('pincho', None), ('de', 'sps00'), ('tortilla', 'ncfs000'), (',', 'Fc'), ('quiero', 'vmip1s0'), ('una', 'di0fs0'), ('pizza', None), ('margherita', None), (',', 'Fc'), ('Quiero', 'vmip1s0'), ('una', 'di0fs0'), ('hamburguesa', None), (',', 'Fc'), ('Podría', None), ('ser', 'vsn0000'), ('un', 'di0ms0'), ('bocadillo', None), ('de', 'sps00'), ('calamares', None), (',', 'Fc'), ('Una', 'di0fs0'), ('ración', None), ('de', 'sps00'), ('patatas', 'ncfp000'), (',', 'Fc'), ('Quería', None), ('pedir', 'vmn0000'), ('dos', 'dn0cp0'), ('cheeseburger', None), (',', 'Fc'), ('una', 'di0fs0'), ('ración', None), ('de', 'sps00'), ('jamón', 'ncms000'), ('ibérico', None), (',', 'Fc'), ('Dos', 'dn0cp0'), ('pizzas', None), ('medianas', 'aq0fp0'), ('barbacoa', None), (',', 'Fc'), ('Un', 'di0ms0'), ('kebab', None), ('mixto', 'aq0ms0'), (',', 'Fc'), ('Chili', None), ('con', 'sps00'), ('queso', None), (',', 'Fc'), ('por', 'sps00'), ('favor', 'ncms000')]
___________

In [8]:
def pedido_cantidad(sentence):

    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    grammar=r'''
    Comida: 
    {<CD>*<NN><IN>*<NN>+} 
    {<CD>*<NN>+}  
    {<JJ>*<NN>+}
     
    Cantidad: {<JJ>}
    {<CD>}
    {<DT>}
    '''
    resultado = RegexpParser(grammar).parse(tagged)
    print(resultado)
    data = []
    for nodo in resultado:
        if type(nodo) == tuple:
          continue
        tipo = nodo.label()
        for elemento in nodo:
          if type(elemento) != tuple:
            continue
          palabra, categoria = elemento
          if tipo == 'Comida':
            data.append(dict(Comida=palabra, Cantidad=1))
               
    return print(data)
   

In [9]:
print ("Bienvenido a ***** ¿Cuál es su pedido?")
ped=input()
pedido_cantidad(ped)



Bienvenido a ***** ¿Cuál es su pedido?
una pizza y bocata
(S (Cantidad una/JJ) (Comida pizza/NN y/NN bocata/NN))
[{'Comida': 'pizza', 'Cantidad': 1}, {'Comida': 'y', 'Cantidad': 1}, {'Comida': 'bocata', 'Cantidad': 1}]
